In [23]:
%pip install python-dotenv langchain langchain-community langchain_groq
%pip install -U langchain_community langchain_anthropic langchain_experimental matplotlib langgraph
%pip install -U duckduckgo-search
%pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.56
    Uninstalling langchain-core-0.3.56:
      Successfully uninstalled langchain-core-0.3.56


In [ ]:
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [25]:
import os
from dotenv import load_dotenv
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.tools import DuckDuckGoSearchRun
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
# Load environment variables
load_dotenv()

False

In [26]:
# run_squad_search_agent.py

from google.colab import drive
import json
from tqdm import tqdm  # For progress tracking
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# === MODEL SETUP ===
llm = ChatOpenAI(
    model="deepseek-reasoner",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_base="https://api.deepseek.com/v1",
    temperature=0.0
)

# === DIRECT LLM QA PROMPT ===
direct_qa_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "You are a SQuAD‑style QA system. Answer the question "
        "using only your internal knowledge—no external tools.\n\n"
        "Context:\n{context}\n\n"
        "Question:\n{question}\n\n"
        "Answer (just the answer string, or empty string if unanswerable):"
    )
)

def generate_final_answer(llm, context: str, question: str) -> str:
    prompt_text = direct_qa_prompt.format(
        context=context.replace('"', '\\"'),
        question=question.replace('"', '\\"')
    )
    return llm.invoke(prompt_text).content.strip()

# === MAIN FLOW ===
if __name__ == "__main__":
    # 1) Mount Google Drive and load the SQuAD v2 dev set
    drive.mount('/content/drive')
    path = "/content/drive/My Drive/DataSet/dev-v2.0.json"
    with open(path, 'r', encoding='utf-8') as f:
        squad = json.load(f)

    # 2) Flatten into a list of (id, context, question) dicts
    examples = []
    for art in squad["data"]:
        for para in art["paragraphs"]:
            for qa in para["qas"]:
                examples.append({
                    "id":       qa["id"],
                    "context":  para["context"],
                    "question": qa["question"]
                })

    # 3) Take up to the first 100 examples
    examples_to_process = examples[:100]

    # 4) Initialize predictions dictionary
    preds = {}

    # 5) Process each example with a progress bar
    for i, ex in enumerate(tqdm(examples_to_process, desc="Processing SQuAD examples")):
        qid      = ex["id"]
        context  = ex["context"]
        question = ex["question"]

        print(f"\n\n=== Processing Question {i+1}/{len(examples_to_process)} ===")
        print(f"ID: {qid}")
        print(f"Question: {question}")

        try:
            # Generate and print the final answer
            final_answer = generate_final_answer(llm, context, question)
            print("\n=== Final Answer ===")
            print(final_answer)

            # Store prediction
            preds[qid] = final_answer

            # Save a checkpoint every 10 questions
            if (i + 1) % 10 == 0:
                ckpt_path = "/content/drive/My Drive/DataSet/preds_checkpoint.json"
                with open(ckpt_path, "w", encoding="utf-8") as f:
                    json.dump(preds, f, ensure_ascii=False, indent=2)
                print(f"Checkpoint saved: {len(preds)} questions processed")

        except Exception as e:
            print(f"Error processing question {qid}: {e}")
            preds[qid] = ""

    # 6) Save final results
    out_path = "/content/drive/My Drive/DataSet/preds.json"
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(preds, f, ensure_ascii=False, indent=2)
    print(f"\nWrote predictions for {len(preds)} questions to {out_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Processing SQuAD examples:   0%|          | 0/100 [00:00<?, ?it/s]



=== Processing Question 1/100 ===
ID: 56ddde6b9a695914005b9628
Question: In what country is Normandy located?


Processing SQuAD examples:   1%|          | 1/100 [00:08<14:50,  8.99s/it]


=== Final Answer ===
France


=== Processing Question 2/100 ===
ID: 56ddde6b9a695914005b9629
Question: When were the Normans in Normandy?


...


=== Final Answer ===
1066


=== Processing Question 99/100 ===
ID: 56de16ca4396321400ee25c8
Question: Who was the ruling class ahead of the Normans?


Processing SQuAD examples:  99%|█████████▉| 99/100 [28:25<00:12, 12.69s/it]


=== Final Answer ===
Anglo-Saxons


=== Processing Question 100/100 ===
ID: 5ad3f4b1604f3c001a3ff951
Question: When did King Harold II conquer England?


Processing SQuAD examples: 100%|██████████| 100/100 [28:44<00:00, 17.25s/it]


=== Final Answer ===
Answer: 

The answer is an empty string because the context states that Duke William II of Normandy conquered England in 1066, defeating and killing King Harold II at the Battle of Hastings. King Harold II did not conquer England; he was the reigning monarch who was defeated. The question contains a factual inaccuracy, making it unanswerable as posed.
Checkpoint saved: 100 questions processed

Wrote predictions for 100 questions to /content/drive/My Drive/DataSet/preds.json


In [27]:
import json, re

# 1. Load your raw preds.json (dict of id → long Markdown)
raw_preds = json.load(open("/content/drive/My Drive/DataSet/preds.json"))

# 2. Extract just the answer span.
#    Here we strip Markdown and take the Conclusion paragraph.
clean_preds = {}
for qid, text in raw_preds.items():
    # Split at the "**Conclusion:**" header (if present)
    parts = text.split("**Conclusion:**")
    answer = parts[-1].strip()      # use everything after that header
    # Remove remaining Markdown bullets, sources, etc.
    answer = re.sub(r"\*|\[SOURCE:.*?\]", "", answer).strip()
    clean_preds[qid] = answer

# Now clean_preds maps id → plain answer text
# 3. Reformat into the list-of-dicts that Evaluate expects
formatted = [
    {"id": qid, "prediction_text": ans, "no_answer_probability": 0.0}
    for qid, ans in clean_preds.items()
]

# 4. (Optional) Persist to disk for inspection
with open("/content/drive/My Drive/DataSet/formatted_preds.json", "w") as f:
    json.dump(formatted, f, indent=2)

In [28]:
# Load your formatted preds
formatted = json.load(open("/content/drive/My Drive/DataSet/formatted_preds.json"))

# Print the total count and the first 3 items
print(f"Total predictions: {len(formatted)}\n")
print(json.dumps(formatted[:3], indent=2))


Total predictions: 100

[
  {
    "id": "56ddde6b9a695914005b9628",
    "prediction_text": "France",
    "no_answer_probability": 0.0
  },
  {
    "id": "56ddde6b9a695914005b9629",
    "prediction_text": "10th and 11th centuries",
    "no_answer_probability": 0.0
  },
  {
    "id": "56ddde6b9a695914005b962a",
    "prediction_text": "Denmark, Iceland, Norway",
    "no_answer_probability": 0.0
  }
]


In [29]:
!pip install evaluate

In [30]:
!python -c "import evaluate; print(evaluate.load('exact_match').compute(references=['a'], predictions=['a']))"

2025-05-05 02:07:51.182107: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746410871.223945   12487 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746410871.235963   12487 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
{'exact_match': np.float64(1.0)}


In [31]:
!git clone https://github.com/rajpurkar/SQuAD-explorer.git
%cd SQuAD-explorer


Cloning into 'SQuAD-explorer'...
remote: Enumerating objects: 5563, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 5563 (delta 11), reused 17 (delta 6), pack-reused 5539 (from 1)
Receiving objects: 100% (5563/5563), 52.26 MiB | 9.23 MiB/s, done.
Resolving deltas: 100% (3563/3563), done.
/content/SQuAD-explorer/SQuAD-explorer/SQuAD-explorer


In [32]:
import evaluate
squad_v2 = evaluate.load("squad_v2")

In [33]:
from prettytable import PrettyTable
dev = json.load(open("/content/drive/My Drive/DataSet/dev-v2.0.json"))
all_refs = []
for art in dev["data"]:
    for para in art["paragraphs"]:
        for qa in para["qas"]:
            all_refs.append({
                "id": qa["id"],
                "answers": {
                    "text": [a["text"] for a in qa["answers"]],
                    "answer_start": [a["answer_start"] for a in qa["answers"]]
                }
            })

# 2. Filter down to only the IDs in your formatted preds
pred_ids = {p["id"] for p in formatted}
filtered_refs = [r for r in all_refs if r["id"] in pred_ids]

# 3. Now lengths match: len(formatted)==len(filtered_refs)==1
from evaluate import load
squad_v2 = load("squad_v2")
results = squad_v2.compute(predictions=formatted, references=filtered_refs)
table = PrettyTable()
table.field_names = ["Metric", "Value"]
table.add_row(["Exact Match (%)", results["exact"]])
table.add_row(["F1 Score (%)"   , results["f1"]])

print(table)

+-----------------+-------------------+
|      Metric     |       Value       |
+-----------------+-------------------+
| Exact Match (%) |        36.0       |
|   F1 Score (%)  | 40.51190476190476 |
+-----------------+-------------------+
